# Additional Examples
This notebook contains more advanced examples using pySSV.

In [ ]:
# Google colab support
try:
    # Try enabling custom widgets, this will fail silently if we're not in Google Colab
    from google.colab import output
    output.enable_custom_widget_manager()
    # Install pySSV for this session
    %pip install pySSV
except:
    pass

### Video
This example takes advantage of the point cloud shader template to render video in real time. In this case, the video is compressed into a quadtree (this is obviously not a very good compression algorithm for video, but it's easy to encode/decode so it makes for a good demonstration) which is stored in a texture. Each row of the animation texture stores the quadtree for one frame where each pixel is one cell in the quadtree. The way this is implemented means the cells don't strictly need to be part of a quad tree, they just each represent a single square of a given size, colour, and location. 

The video quadtree is created by a separate program the code for which can be found here: https://github.com/space928/badapple-quadtree-encoder

In [ ]:
import os.path

# Download the compressed video file from the internet if needed (with the user's permission)
filename = "badapple_quad.pkl"
if not os.path.isfile(filename):
    if input("Encoded video file not found! Do you want to download it now (yes/no)?")[0] == "y":
        url = "https://github.com/space928/badapple-quadtree-encoder/releases/download/0.1.0/badapple_quad.pkl"
        import urllib.request
        try:
            print("Downloading...")
            urllib.request.urlretrieve(url, filename)
            print("Successfully downloaded encoded video file!")
        except Error as e:
            print(f"Failed to download video: {e}")
else:
    print(f"Video file '{filename}' already exists, using existing version...")

In [ ]:
import pySSV as ssv
import numpy as np
import pickle as pkl

canvas5 = ssv.canvas(use_renderdoc=True)
with open("badapple_quad.pkl", "rb") as f:
    anim, frame_lengths = pkl.load(f)
    print(f"Loaded animation! Animation has shape:{anim.shape}")
    
canvas5.main_render_buffer.full_screen_vertex_buffer.update_vertex_buffer(np.zeros((anim.shape[0]*6), dtype=np.float32))

anim = np.swapaxes(anim, 0, 1)
# Dcelare textures, make sure that these textures as treated as ints instead of floats
anim_tex = canvas5.texture(anim, "uAnimTex", treat_as_normalized_integer=False)
frame_lengths_tex = canvas5.texture(frame_lengths, "uFrameLengthsTex", treat_as_normalized_integer=False)
# Setup texture samplers
anim_tex.repeat_x, anim_tex.repeat_y = False, False
anim_tex.linear_filtering = False
frame_lengths_tex.repeat_x, frame_lengths_tex.repeat_y = False, False
frame_lengths_tex.linear_filtering = False

canvas5.shader("""
#pragma SSV point_cloud mainPoint --non_square_points
// These are automatically declared by the preprocessor
//uniform isampler2D uAnimTex;
//uniform isampler2D uFrameLengthsTex;
VertexOutput mainPoint()
{
    VertexOutput o;
    // Synchronise the playback to the time uniform, 30 FPS
    int frame = int(uTime*30.-20.);
    
    int frameLen = texelFetch(uFrameLengthsTex, ivec2(0, frame), 0).r;
    if(gl_VertexID > frameLen) 
    {
        // Early out for verts not needed in this frame; no geometry will be generated for these as the size is set to 0
        o.size = vec2(0.);
        return o;
    }
    // This contains the data for the current quad to rendered (value (0-255), x (pixels), y (pixels), subdivision (0-n))
    ivec4 quad = texelFetch(uAnimTex, ivec2(gl_VertexID, frame), 0);
    // The size is determined by the subdivision level of the cell in the quad tree. 
    o.size = vec2(1./pow(2., quad.w-0.1));
    if(quad.w == 0)
        o.size = vec2(0.);
    vec4 pos = vec4(float(quad.z)/480., 1.-float(quad.y)/360., 0., 1.);
    pos.xy += o.size/vec2(2., -2.);  // Centre the point
    pos = pos*2.-1.;  // To clip space (-1 to 1)
    pos += vec4(in_vert, 0.)*1e-8;  // If in_vert is not used, the shader compiler optimises it out which makes OpenGL unhappy; this may be fixed in the future
    o.position = pos;
    o.color = vec4(vec3(float(quad.x)/255.0)+in_color, 1.0);
    return o;
}
""")
canvas5.run()

### Geometry shaders
This shader demonstrates the use of custom geometry shaders to render a vector field.

In [ ]:
import pySSV as ssv
import numpy as np

# Generate some points
def generate_points():
    width, depth = 64, 64
    scale = 3
    v_scale = 0.5
    f = 0.01
    verts = np.zeros((width, depth, 9), dtype='f4')
    for z in range(depth):
        for x in range(width):
            dx = width/2 - x
            dz = depth/2 - z
            y = np.sin((dx*dx+dz*dz)*f) * v_scale
            # Pos
            verts[z, x, :3] = [x/width * scale, y, z/depth * scale]
            # Colour
            verts[z, x, 3:6] = [y/v_scale, abs(y/v_scale), np.sin(y/v_scale*10.)*0.5+0.5]
            # Direction
            verts[z, x, 6:9] = [dx/width, 0.1, dz/depth]
            
    return verts.flatten()

canvas5 = ssv.canvas(use_renderdoc=True)
# Set the contents of default vertex buffer on the main pass (normally used for full-screen shaders, but in this case hijacked for this example)
canvas5.main_render_buffer.full_screen_vertex_buffer.update_vertex_buffer(generate_points(), ("in_vert", "in_color", "in_dir"))
canvas5.main_camera.target_pos = np.array((1.5, 0, 1.5))
#print(canvas5.dbg_preprocess_shader("""
canvas5.shader("""
#pragma SSV geometry mainPoint mainGeo --vertex_output_struct VertexOutput --geo_max_vertices 7 --custom_vertex_input
struct VertexOutput {
    vec4 position;
    vec4 color;
    vec3 dir;
    float size;
};

#ifdef SHADER_STAGE_VERTEX
in vec3 in_vert;
in vec3 in_color;
in vec3 in_dir;

VertexOutput mainPoint()
{
    VertexOutput o;
    vec4 pos = vec4(in_vert, 1.0);
    //pos = uViewMat * pos;
    //pos = uProjMat * pos;
    o.position = pos;
    o.color = vec4(in_color, 1.);
    o.size = 30.0/uResolution.x;
    o.dir = normalize(in_dir);
    return o;
}
#endif

#ifdef SHADER_STAGE_GEOMETRY
void mainGeo(VertexOutput i) {
    vec4 position = i.position;
    float size = i.size;
    // This output variable is defined by the template and must be written to before the first EmitVertex() call to take effect
    out_color = i.color;
    vec3 fwd = normalize((uViewMat * vec4(0., 0., 1., 0.)).xyz);
    vec3 perp = normalize(cross(i.dir, fwd));
    vec4 aspect_ratio = vec4(1., uResolution.x/uResolution.y, 1., 1.);
    float baseWidth = 0.05;
    float headWidth = 0.2;
    float headLength = 0.4;
    // Now we draw an arrow
    // Base
    out_color = vec4(0.,0.,0.,1.);
    gl_Position = position + size * vec4(perp*baseWidth, 0.0) * aspect_ratio;
    gl_Position = uProjMat * uViewMat * gl_Position;
    EmitVertex();
    gl_Position = position + size * vec4(-perp*baseWidth, 0.0) * aspect_ratio;
    gl_Position = uProjMat * uViewMat * gl_Position;
    EmitVertex();
    out_color = i.color;
    gl_Position = position + size * vec4(i.dir + perp*baseWidth, 0.0) * aspect_ratio;
    gl_Position = uProjMat * uViewMat * gl_Position;
    EmitVertex();
    gl_Position = position + size * vec4(i.dir - perp*baseWidth, 0.0) * aspect_ratio;
    gl_Position = uProjMat * uViewMat * gl_Position;
    EmitVertex();
    EndPrimitive();
    // Head
    gl_Position = position + size * vec4(i.dir + perp*headWidth, 0.0) * aspect_ratio;
    gl_Position = uProjMat * uViewMat * gl_Position;
    EmitVertex();
    gl_Position = position + size * vec4(i.dir + -perp*headWidth, 0.0) * aspect_ratio;
    gl_Position = uProjMat * uViewMat * gl_Position;
    EmitVertex();
    gl_Position = position + size * vec4(i.dir * (1.+headLength), 0.0) * aspect_ratio;
    gl_Position = uProjMat * uViewMat * gl_Position;
    EmitVertex();
    EndPrimitive();
}
#endif
""")#)
canvas5.run(stream_quality=100)

### Streaming Modes
*pySSV* supports a number of different video streaming modes to get rendered frames from the OpenGL backend into Jupyter. They each have their own advantages and disadvantages, so you can experiment with which method works best for you. `JPG` should be supported everywhere, but if your platform supports it, I would recommend `VP8` or `MJPEG`.

Not all streaming modes are supported on all platforms. Google Colab is notoriously difficult to get working nicely.

Here we present a particularly difficult example for video encoders, a point cloud (taken from the introduction.ipynb notebook) and how the different encoding settings affect it.

The following streaming modes are supported:

 - JPG
 - PNG
 - VP8
 - VP9
 - H264
 - MJPEG

The streaming mode is controlled using the `stream_mode` parameter of the `canvas.run()` method which accepts a `str` or an `SSVStreamingMode` (`from pySSV.ssv_render_process_server import SSVStreamingMode`). The `run()` method also takes a `stream_quality` parameter which can be used to control the compression of the encoder. It accepts a value from 0-100 (some encoders will work with values greater than 100, others clamp it) which, depending on the encoder, is scaled to give the constant bit rate or quality factor. Higher values give better quality images at the cost of higher bandwidth utilisation. When the `stream_quality` is above or equal to 90, chroma subsampling is disabled for formats that support yuv444p.

#### Technical Details
Internally, *pySSV* opens a dedicated websocket with the Jupyter frontend to stream video. On platforms where this isn't supported (notably, Google Colab) this falls back to using Jupyter Widget messages which are a bit less efficient due to the protocol's need to json encode everything. The `MJPEG` format is an exception to this as it communicates using a local HTTP server, relying on the browser's native support for `MJPEG` over HTTP; this has the advantage that `MJPEG` frames don't need to be json encoded or parsed in JS which helps a lot with latency.

The image formats `JPG` and `PNG` are encoded using Pillow as base64 encoded data URLs which are passed to an `<img>`. Whereas as the video formats are encoded by libavformat (FFmpeg's encoding library) and decoded in javascript using the WebCodecs API and blitted to a canvas; hence the lack of support for Firefox for these formats. `MJPEG` is encoded by libavformat and passed directly as a URL to the local HTTP server to an `<img>`.

In [ ]:
import pySSV as ssv
import numpy as np

# Make the canvas size a bit bigger to put a bit more pressure on the encocers
CANVAS_SIZE = (1280, 720)

# Generate some points
def generate_points():
    width, depth = 64, 64
    scale = 3
    v_scale = 0.5
    f = 0.01
    verts = np.zeros((width, depth, 6), dtype='f4')
    for z in range(depth):
        for x in range(width):
            dx = width/2 - x
            dz = depth/2 - z
            y = np.sin((dx*dx+dz*dz)*f) * v_scale
            verts[z, x, :3] = [x/width * scale, y, z/depth * scale]
            verts[z, x, 3:6] = [y/v_scale, abs(y/v_scale), np.sin(y/v_scale*10.)*0.5+0.5]
            
    return verts.flatten()

def make_canvas():
    canvas = ssv.canvas(use_renderdoc=True, size=CANVAS_SIZE)
    # Set the contents of default vertex buffer on the main pass (normally used for full-screen shaders, but in this case hijacked for this example)
    canvas.main_render_buffer.full_screen_vertex_buffer.update_vertex_buffer(generate_points())
    canvas.main_camera.target_pos = np.array((1.5, 0, 1.5))
    canvas.shader("""
    #pragma SSV point_cloud mainPoint
    VertexOutput mainPoint()
    {
        VertexOutput o;
        vec4 pos = vec4(in_vert, 1.0);
        pos = uViewMat * pos;
        pos = uProjMat * pos;
        o.position = pos;
        o.color = vec4(in_color, 1.);
        o.size = 30.0/uResolution.x;
        float d = length(uMouse/uResolution.xy*2.-1.-pos.xy/pos.z);
        o.size += clamp(pow(smoothstep(.5, 0., d), 3.)*0.03, 0., 0.3);
        o.color += step(d, o.size);
        return o;
    }
    """)
    return canvas

The default settings when `canvas.run()` is called are `stream_mode="jpg"` and `stream_quality=75`. When `stream_quality` is unset it defaults to the encoder's default quality.

In [ ]:
make_canvas().run()

In [ ]:
# For JPG streams, setting the stream quality to 100 can actually *improve* encoding performance (if not limited by bandwidth) as some of the optimisations can be skipped.
make_canvas().run(stream_quality=100)

In [ ]:
# For PNG is always lossless so the stream quality can't be controlled.
# This format is currently the only one which supports transparency in the output.
# It's also VERY slow to encode.

# With streaming formats the produce very large frames, such as png, Jupyter/the web browser can 
# get backed up with frames, in this case the frame rate may still be reasonable, but extremely 
# high latency (and memory usage!) will be apparent. In this case you need to switch to a streaming 
# format that offers more compression or decrease the streaming quality.

make_canvas().run(stream_mode="png")

#### Video Formats

In [ ]:
# VP8 offers a good balance between quality and encoding time while offering very good compression
# Latency is also generally fairly low
make_canvas().run(stream_mode="vp8", stream_quality=100)

In [ ]:
# VP9 has improved compression efficiency but is much slower at encoding
make_canvas().run(stream_mode="vp9", stream_quality=10)

In [ ]:
# H264 is fast to encode, but the compressions isn't quite as efficient as VP8
make_canvas().run(stream_mode="h264", stream_quality=100)

In [ ]:
# MJPEG has very low latency and fast encoding/decooding time, but worse compression efficiency than other video formats.
make_canvas().run(stream_mode="mjpeg", stream_quality=100)